In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument("--headless")  # run in background
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1920,1080")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)
driver.get("https://www.amazon.com/s?k=laptop")

# Give page time to load
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

titles = soup.select("h2 a span")

for t in titles[:10]:  # Print first 10 titles
    print(t.text)

driver.quit()

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

# -------- Functions to extract data --------
def get_title(soup):
    try:
        title = soup.find("span", id="productTitle")
        title_string = title.text.strip()
    except AttributeError:
        title_string = ""
    return title_string

def get_price(soup):
    try:
        price = soup.find("span", class_="a-offscreen").text.strip()
    except AttributeError:
        price = ""
    return price

def get_rating(soup):
    try:
        rating = soup.find("span", class_="a-icon-alt").text.strip()
    except AttributeError:
        rating = ""
    return rating

#def get_review_count(soup):
 #   try:
  #      review_count = soup.find("span", class_="a-size-base s-underline-text").text.strip()
   # except AttributeError:
    #    review_count = ""
    #return review_count


def get_review_count(soup):
    try:
        # Option 1: By ID (Most reliable if it exists)
        review_count = soup.find("span", id="acrCustomerReviewText")
        if review_count:
            return review_count.text.strip()
        
        # Option 2: By a specific class and role (Another common pattern)
        review_count = soup.find("span", {"data-hook": "total-review-count"})
        if review_count:
            return review_count.text.strip()
            
        # Option 3: Fallback - look for the anchor tag near the ratings
        # This finds the link that contains the number of reviews
        review_link = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})
        if review_link:
            return review_link.text.strip().replace(' ratings', '').replace(' rating', '') # Clean the text
        else:
            return ""
    except AttributeError:
        return ""








def get_availability_status(soup):
    try:
        available = soup.find("div", id="availability").get_text(strip=True)
    except AttributeError:
        available = "Not Available"
    return available

# Function to extract Image URL
def get_image_url(soup):
    try:
        image = soup.find("img", id="landingImage")
        return image["src"]
    except (AttributeError, TypeError):
        return ""


# -------- Main Script --------
# -------- Main Script --------
if __name__ == '__main__':
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36", 
        "Accept-Language": "en-US, en;q=0.9"
    }

    BASE_URL = "https://www.amazon.com/s?k=planter+pots&crid=2VUYO3PT77H21&sprefix=planter+pot%2Caps%2C514&ref=nb_sb_noss_2&page={page}"
    
    d = {"title": [], "price": [], "rating": [], "reviews": [], "availablity": [], "image_url": []}

    # 🔥 Scrape first 5 pages
    for page in range(1, 7):
        print(f"Scraping page {page}...")
        URL = BASE_URL.format(page=page)
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")

        # Extract product links
        links = soup.find_all("a", class_="a-link-normal s-no-outline")
        link_list = [link.get("href") for link in links if link.get("href")]

        # Visit each product page
        for link in link_list:
            if link.startswith("http"):
                url = link
            else:
                url = "https://www.amazon.com" + link

            try:
                new_webpage = requests.get(url, headers=HEADERS)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")

                d['title'].append(get_title(new_soup))
                d['price'].append(get_price(new_soup))
                d['rating'].append(get_rating(new_soup))
                d['reviews'].append(get_review_count(new_soup))
                d['availablity'].append(get_availability_status(new_soup))
                d['image_url'].append(get_image_url(new_soup))  # ✅ new field

                time.sleep(1)
            except Exception as e:
                print(f"❌ Skipped {url} due to error: {e}")

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_planter_pots.csv", header=True, index=False)

    print("✅ Scraping completed. Data saved to amazon_planter_pots.csv")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
✅ Scraping completed. Data saved to amazon_planter_pots.csv


In [1]:
amazon_df

NameError: name 'amazon_df' is not defined

# Data Cleaning

In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("amazon_planter_pots.csv")

# 1. Remove duplicates
df.drop_duplicates()

,title,price,rating,reviews,availablity,image_url
0,Indoor Self Watering Planters with Drainage Ho...,$19.99,4.6 out of 5 stars,"3,546 ratings",In Stock,https://m.media-amazon.com/images/I/719MB0Tuw7...
1,Set of 2 Large Tall Planter for Indoor or Outd...,$62.99,4.7 out of 5 stars,502 ratings,In Stock,https://m.media-amazon.com/images/I/714Y3Y1I9B...
2,Anyumocz 60 Pcs 1 Gallon Flexible Plant Nurser...,$14.99,4.4 out of 5 stars,573 ratings,In Stock,https://m.media-amazon.com/images/I/8143GuNSe6...
3,"MIXC 30 Packs 4"" Reinforced Clear Nursery Pots...",$20.99,4.8 out of 5 stars,740 ratings,In Stock,https://m.media-amazon.com/images/I/71gyI7G5ng...
5,"Quarut Large Plastic 3 Pack16 inch Plant Pots,...",$49.38,4.6 out of 5 stars,"6,040 ratings",In Stock,https://m.media-amazon.com/images/I/81m+QmXzU2...
...,...,...,...,...,...,...
293,"4 Inch Herb Planter Pot, Plastic Planter with ...",$33.99,4.7 out of 5 stars,197 ratings,In Stock,https://m.media-amazon.com/images/I/81Mmad3se4...
294,QCQHDU Set of 2 Oversized 9 Gallon Round Plant...,$87.00,4.5 out of 5 stars,78 ratings,In Stock,https://m.media-amazon.com/images/I/81mBxbZDtQ...
295,"Smiley Face Planter Pot, Cute Coffee Plant Pot...",$15.89,4.6 out of 5 stars,201 ratings,In Stock,https://m.media-amazon.com/images/I/71ApNHZFf6...
296,"LaLaGreen Bundle- 2 Pack, 10 Inch Hanging Bask...",$23.99,4.2 out of 5 stars.,NaN,Currently unavailable.We don't know when or if...,https://m.media-amazon.com/images/I/71bQ5stR54...


In [5]:
import numpy as np
# Clean Price column in-place
df['price'] = (
    df['price']
    .astype(str)                               # make sure it's string
    .str.replace(r'[^0-9.]', '', regex=True)   # keep only numbers + dot
    .replace('', np.nan)                       # empty -> NaN
)
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # convert to float

# Clean Rating column in-place
df['rating'] = (
    df['rating']
    .astype(str)
    .str.extract(r'([\d\.]+)', expand=False)   # extract number
)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')



# Clean Reviews column in-place
df['reviews'] = (
    df['reviews']
    .astype(str)                               # Ensure it's a string
    .str.replace('ratings', '', case=False)    # Remove the word 'ratings' (case-insensitive)
    .str.replace('reviews', '', case=False)    # Remove the word 'reviews' (case-insensitive)
    .str.replace(',', '')                      # Remove commas for thousands
    .str.strip()                               # Remove any leading/trailing whitespace
    .replace('', np.nan)                       # Empty strings become NaN
)
# Convert the cleaned string to a numeric value (integer)
df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce', downcast='integer')



# Clean Title (optional: remove extra spaces)
df['title'] = df['title'].astype(str).str.strip()

# Drop rows with missing key values
df = df.dropna(subset=['title', 'price', 'rating'])

# Save cleaned dataset
df.to_csv("amazon_planter_pots_clean.csv", index=False)

df


,title,price,rating,reviews,availablity,image_url
0,Indoor Self Watering Planters with Drainage Ho...,19.99,4.6,3546.0,In Stock,https://m.media-amazon.com/images/I/719MB0Tuw7...
1,Set of 2 Large Tall Planter for Indoor or Outd...,62.99,4.7,502.0,In Stock,https://m.media-amazon.com/images/I/714Y3Y1I9B...
2,Anyumocz 60 Pcs 1 Gallon Flexible Plant Nurser...,14.99,4.4,573.0,In Stock,https://m.media-amazon.com/images/I/8143GuNSe6...
3,"MIXC 30 Packs 4"" Reinforced Clear Nursery Pots...",20.99,4.8,740.0,In Stock,https://m.media-amazon.com/images/I/71gyI7G5ng...
4,"MIXC 30 Packs 4"" Reinforced Clear Nursery Pots...",20.99,4.8,740.0,In Stock,https://m.media-amazon.com/images/I/71gyI7G5ng...
...,...,...,...,...,...,...
295,"Smiley Face Planter Pot, Cute Coffee Plant Pot...",15.89,4.6,201.0,In Stock,https://m.media-amazon.com/images/I/71ApNHZFf6...
296,"LaLaGreen Bundle- 2 Pack, 10 Inch Hanging Bask...",23.99,4.2,NaN,Currently unavailable.We don't know when or if...,https://m.media-amazon.com/images/I/71bQ5stR54...
297,"Flower Pots, 8 Inch Plastic Plant Pots for Pla...",23.99,3.7,7.0,In Stock,https://m.media-amazon.com/images/I/617FPc2T5i...
298,"vancasso Ceramic Plant Pots, 4.25+5.25+6.25 In...",23.99,4.6,71.0,In Stock,https://m.media-amazon.com/images/I/71fiwAlEX5...


In [7]:
import pandas as pd

# Example keyword list
keywords = ["ceramic", "plastic", "self-watering", "metal", "resin", "hanging", "indoor", "outdoor"]

# Function to extract ALL matching keywords
def extract_keywords(title):
    title_lower = str(title).lower()
    matched = [word for word in keywords if word in title_lower]
    return ", ".join(matched) if matched else "other"

# Apply function
df["keywords"] = df["title"].apply(extract_keywords)

# Split into multiple rows (useful for analysis in SQL/Power BI)
df_exploded = df.assign(keyword=df["keywords"].str.split(", ")).explode("keyword")

# Convert list of keywords into a comma-separated string
df["keywords"] = df["keywords"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))


# Check result
#print(df_exploded[["title", "keyword", "price"]].head(10))
df

,title,price,rating,reviews,availablity,image_url,keywords
0,Indoor Self Watering Planters with Drainage Ho...,19.99,4.6,3546.0,In Stock,https://m.media-amazon.com/images/I/719MB0Tuw7...,indoor
1,Set of 2 Large Tall Planter for Indoor or Outd...,62.99,4.7,502.0,In Stock,https://m.media-amazon.com/images/I/714Y3Y1I9B...,"indoor, outdoor"
2,Anyumocz 60 Pcs 1 Gallon Flexible Plant Nurser...,14.99,4.4,573.0,In Stock,https://m.media-amazon.com/images/I/8143GuNSe6...,plastic
3,"MIXC 30 Packs 4"" Reinforced Clear Nursery Pots...",20.99,4.8,740.0,In Stock,https://m.media-amazon.com/images/I/71gyI7G5ng...,plastic
4,"MIXC 30 Packs 4"" Reinforced Clear Nursery Pots...",20.99,4.8,740.0,In Stock,https://m.media-amazon.com/images/I/71gyI7G5ng...,plastic
...,...,...,...,...,...,...,...
295,"Smiley Face Planter Pot, Cute Coffee Plant Pot...",15.89,4.6,201.0,In Stock,https://m.media-amazon.com/images/I/71ApNHZFf6...,"indoor, outdoor"
296,"LaLaGreen Bundle- 2 Pack, 10 Inch Hanging Bask...",23.99,4.2,NaN,Currently unavailable.We don't know when or if...,https://m.media-amazon.com/images/I/71bQ5stR54...,"metal, hanging, outdoor"
297,"Flower Pots, 8 Inch Plastic Plant Pots for Pla...",23.99,3.7,7.0,In Stock,https://m.media-amazon.com/images/I/617FPc2T5i...,"plastic, indoor, outdoor"
298,"vancasso Ceramic Plant Pots, 4.25+5.25+6.25 In...",23.99,4.6,71.0,In Stock,https://m.media-amazon.com/images/I/71fiwAlEX5...,"ceramic, indoor, outdoor"


# Connecting to MySQL Workbench


In [10]:
#pip install mysql-connector-python sqlalchemy
import mysql.connector as connection
from sqlalchemy import create_engine

#connecting to MySQL
mydb = connection.connect(host="localhost", user="put your user", passwd="password", use_pure=True)

In [11]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
from urllib.parse import quote_plus

# Your cleaned dataframe
# df = ...

df['scrape_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# MySQL connection details
MYSQL_USER = "Username"
MYSQL_PASSWORD = "Your Password"   # password with @ symbol
MYSQL_HOST = "localhost"
MYSQL_PORT = "3306"
MYSQL_DB = "Test"

# URL-encode the password to handle special characters
encoded_password = quote_plus(MYSQL_PASSWORD)

# ✅ Correct connection string with encoded password
engine = create_engine(
    f"mysql+mysqlconnector://{MYSQL_USER}:{encoded_password}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
)

# Insert into SQL
df.to_sql(
    name="planter_pots",
    con=engine,
    if_exists="append",   # append rows
    index=False
)

print("✅ Data inserted into MySQL successfully.")

✅ Data inserted into MySQL successfully.
